In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import json
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown

def load_stock_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read()
        content = content.replace("'", '"')
        stock_info = json.loads(content)
    return stock_info
    
def get_stock_data(stock_code, start_date, end_date):
    stock_url = f"https://finance.naver.com/item/sise_day.naver?code={stock_code}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    stock_data = []
    page = 1

    while True:
        page_url = f"{stock_url}&page={page}"
        response = requests.get(page_url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"class": "type2"})

        rows = table.find_all("tr")[1:]
        for row in rows:
            cols = row.find_all("td")
            if len(cols) < 7:
                continue

            date_str = cols[0].text.strip()
            price_str = cols[1].text.strip().replace(",", "")
            if date_str and price_str.isdigit():
                date = datetime.strptime(date_str, "%Y.%m.%d")
                if date < start_date:
                    break
                if date <= end_date:
                    price = int(price_str)
                    stock_data.append((date, price))

        if date < start_date:
            break
        page += 1

    df = pd.DataFrame(stock_data, columns=["Date", "Price"])
    df = df.sort_values("Date").reset_index(drop=True)
    return df

def calculate_market_cap(stock_data, listed_shares):
    stock_data["Market Cap"] = stock_data["Price"] * listed_shares / 10**12  # 시가총액을 조원 단위로 변환
    stock_data["Market Cap"] = stock_data["Market Cap"].round(2)  # 소수점 둘째 자리까지 반올림
    return stock_data[["Date", "Market Cap"]]

def calculate_price_change(stock_data, base='start'):
    if base == 'start':
        base_price = stock_data.iloc[0]["Price"]
    elif base == 'end':
        base_price = stock_data.iloc[-1]["Price"]
    else:
        raise ValueError("Invalid base value. Use 'start' or 'end'.")

    stock_data["Price Change"] = (stock_data["Price"] - base_price) / base_price * 100
    return stock_data[["Date", "Price Change"]]


def main(selected_stock):
    plt.rcParams["font.family"] = 'Malgun Gothic'
    plt.rcParams['axes.unicode_minus'] = False
    
    filepath = "top100list.txt"
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2024, 6, 1)

    stock_info = load_stock_info(filepath)

    stock_name = selected_stock
    stock_code = stock_info[stock_name]["code"]
    listed_shares = int(stock_info[stock_name]["listed_shares"])

    stock_data = get_stock_data(stock_code, start_date, end_date)
    
    if "split_date" in stock_info[stock_name] and "split_ratio" in stock_info[stock_name]:
        split_date = datetime.strptime(stock_info[stock_name]["split_date"], "%Y-%m-%d")
        split_ratio = stock_info[stock_name]["split_ratio"]
        stock_data.loc[stock_data["Date"] < split_date, "Price"] /= split_ratio

    market_cap_data = calculate_market_cap(stock_data, listed_shares)
    price_change_data_start = calculate_price_change(stock_data, base='start')
    price_change_data_end = calculate_price_change(stock_data, base='end')

    fig, axs = plt.subplots(4, figsize=(10, 15))

    # Plot stock price
    axs[0].plot(stock_data["Date"], stock_data["Price"], label="Stock Price", color="blue")
    axs[0].set_ylabel("Stock Price")
    axs[0].grid(True)
    axs[0].legend(loc="upper left")

    # Plot market cap data
    axs[1].plot(market_cap_data["Date"], market_cap_data["Market Cap"], label="Market Cap", color="green")
    axs[1].set_ylabel("Market Cap")
    axs[1].grid(True)
    axs[1].legend(loc="upper left")

    # Plot price change from start date
    axs[2].plot(price_change_data_start["Date"], price_change_data_start["Price Change"], label="Price Change (Start Date)", color="red")
    axs[2].set_ylabel("Price Change (%)")
    axs[2].grid(True)
    axs[2].legend(loc="upper left")

    # Plot price change from end date
    axs[3].plot(price_change_data_end["Date"], price_change_data_end["Price Change"], label="Price Change (End Date)", color="orange")
    axs[3].set_ylabel("Price Change (%)")
    axs[3].grid(True)
    axs[3].legend(loc="upper left")

    plt.suptitle(f"Stock Analysis for {stock_name}")
    plt.show()

# Dropdown을 통해 주식 선택
stock_list = list(load_stock_info("top100list.txt").keys())
interact(main, selected_stock=Dropdown(options=stock_list, description='Stock:'))

interactive(children=(Dropdown(description='Stock:', options=('삼성전자', 'SK하이닉스', 'LG에너지솔루션', '현대차', '삼성바이오로직스',…

<function __main__.main(selected_stock)>